In [35]:
#Here is the tutorial this is based on: https://plotly.com/python/choropleth-maps/

#load packages
import plotly.graph_objects as go
import pandas as pd

In [43]:
#render fix
import plotly.io as pio
pio.renderers.default='notebook'

In [44]:
#load and preview data frame
df = pd.read_csv('/Users/carakarter/mRelief_Data_Projects/Map_State_SNAP_Participation_Rates/Eligible_not_participating_by_state_fy19.csv')
df.head()

,Eligible_thousands,State,Code,Significant,Lower_B,Rate,Upper_B,Not_participating_thousands,Not_participating_actual
0,859,Alabama,AL,NaN,75,80,84,171.8,171800
1,89,Alaska,AK,*,82,89,96,9.79,9790
2,910,Arizona,AZ,*,71,76,82,218.4,218400
3,536,Arkansas,AR,*,59,64,69,192.96,192960
4,"4,787",California,CA,*,68,70,73,"1,436",1436100


In [66]:
#create visualization for total N
fig = go.Figure(data=go.Choropleth(
    locations=df['Code'], # Spatial coordinates
    z = df['Not_participating_actual'].astype(float), # Data to be color-coded
    locationmode = 'USA-states', # set of locations match entries in `locations`
    colorscale = 'Reds',
    colorbar_title = "Estimated Eligible Individuals not Enrolled"
))

fig.update_layout(
    title_text = 'FY2019 Individuals Eligible but Not Participating in SNAP',
    geo_scope='usa', # limit map scope to USA
    annotations = [dict(
        x=0,
        y=0,
        xref='paper',
        yref='paper',
        text='Source: <a href="https://fns-prod.azureedge.us/sites/default/files/resource-files/snap-tech-partrate2017-2019.pdf">\
            FY19 SNAP Participation Estimates</a>',
        showarrow = False
    )]
)


fig.show()

In [67]:
#create visualization for total N
fig = go.Figure(data=go.Choropleth(
    locations=df['Code'], # Spatial coordinates
    z = df['Rate'].astype(float), # Data to be color-coded
    locationmode = 'USA-states', # set of locations match entries in `locations`
    colorscale = 'Greens',
    colorbar_title = "Participation Rate"
))

fig.update_layout(
    title_text = 'FY2019 Rate of Eligible Individuals Enrolled in SNAP',
    geo_scope='usa', # limit map scope to USA
        annotations = [dict(
        x=0,
        y=0,
        xref='paper',
        yref='paper',
        text='Source: <a href="https://fns-prod.azureedge.us/sites/default/files/resource-files/snap-tech-partrate2017-2019.pdf">\
            FY19 SNAP Participation Estimates</a>',
        showarrow = False
    )]
)

fig.show()